In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.graph_objs as go
import plotly
import plotly.subplots as sp
import plotly.io as pio
import os
from cloudvolume import CloudVolume as cv
from caveclient import CAVEclient
import seaborn as sns 
import scipy.sparse
import meshparty
import pygsheets

vol =cv('graphene://https://minnie.microns-daf.com/segmentation/table/zheng_ca3', agglomerate=True, use_https=True)
client = CAVEclient('zheng_ca3')

isotropy = [5,5,2]


In [ ]:
gc = pygsheets.authorize(service_file = "./hippca3-8126bea0d603.json")
sheet = gc.open('all_pyramidal_cells')
worksheet = sheet.worksheet('title','all_pyr_v2_remove_dupes')
pc_df = worksheet.get_as_df()
str_columns = worksheet.get_as_df(numerize=False)['nul_segids']
pc_df['nul_segids'] = str_columns

In [ ]:
# update all_pyr_v2_remove_dupes sheet from CA3 proofreading sheets 1.pyramidal_cell_list, 6.Additional_pyramidal

sheet2 = gc.open('CA3 proofreading')
worksheet2 = sheet2.worksheet('title','1.pyramidal_cell_list')
pc_df2 = worksheet2.get_as_df(start='A9')
str_columns = worksheet2.get_as_df(start='A9', numerize=False)['segid (Nuclei table segID)']
pc_df2['segid (Nuclei table segID)'] = str_columns

worksheet3 = sheet2.worksheet('title','6.Additional_pyramidal')
pc_df3 = worksheet3.get_as_df(start='A9')
str_columns = worksheet3.get_as_df(start='A9', numerize=False)['segid (Nuclei table segID)']
pc_df3['segid (Nuclei table segID)'] = str_columns

for i in range(0, len(pc_df)):
    nuc_seg_id = pc_df['nul_segids'][i]
    row_idx2 = np.where(nuc_seg_id == pc_df2['segid (Nuclei table segID)'].values)[0]
    if row_idx2.size == 1: 
        pc_df.loc[i, 'Final Link'] = pc_df2['Final Link'].values[row_idx2][0] 
    else:
        row_idx3 = np.where(nuc_seg_id == pc_df3['segid (Nuclei table segID)'].values)[0]
        pc_df.loc[i, 'Final Link'] = pc_df3['Final Link'].values[row_idx3][0] 
    

In [ ]:
nuclei_coord_in_vol = pc_df[['x','y','z']].to_numpy()
latest_roots_PC = []
print('Getting latest roots for PC...')  
cell_ids_scatter = vol.scattered_points(nuclei_coord_in_vol)
for i in range(0, len(nuclei_coord_in_vol)):
    if i % 100 == 0:
        print(i)
    cell_ID = [cell_ids_scatter.get(tuple(nuclei_coord_in_vol[i,:]))]
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0]-150, nuclei_coord_in_vol[i,1], nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0], nuclei_coord_in_vol[i,1]+150, nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0], nuclei_coord_in_vol[i,1]-150, nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        print("Failed to find neuron segment ID using nuclei center")
    else:
        if client.chunkedgraph.is_latest_roots(cell_ID[0]):
            latest_roots_PC.append(cell_ID[0])
        else:
            latest_roots_PC.append(client.chunkedgraph.suggest_latest_roots(cell_ID[0]))
latest_roots_PC_str = [[str(num)] for num in latest_roots_PC]


In [ ]:
print(latest_roots_PC_str[0:10])

final_links_matrix = []
for i in range(0,len(pc_df)):
    final_links_matrix.append([pc_df['Final Link'].values[i]])

#worksheet.update_values('P2', latest_roots_PC_str)
worksheet.update_values('AC2', final_links_matrix)

In [ ]:
worksheet.update_values('AC2', final_links_matrix)

In [ ]:
# L2 skeleton

import pcg_skel
import meshparty
import time
from nglui import parser

pyc_i_with_ais = []
ais_all = []
nuc_coord_all = []
ais_error= []

for i in range(0, len(latest_roots_PC)):
    if i not in [190,284,883]:
        link = pc_df['Final Link'][i]
        time.sleep(0.2)
        if link != '':
            this_state_id = link.split('/')[-1]
            json_info = client.state.get_state_json(state_id =this_state_id)
            layerNames = parser.layer_names(json_info)
            ais_tab = [i for i, s in enumerate(layerNames) if "AIS" in s]
            try:
                pts, tags = parser.point_annotations(json_info, layerNames[ais_tab[0]], tags=True)
            except:
                ais_error.append(i+2)
                continue        
                
            if pts != []:
                pyc_i_with_ais.append(i)
                ais = np.array([pts[0][0] /18 * 4, pts[0][1] /18 * 4, pts[0][2] /45 * 40])* np.array([0.018,0.018,0.045])
                ais_all.append(ais)
                nuc_coord = np.array(nuclei_coord_in_vol[i,:])* np.array([0.018,0.018,0.045])
                nuc_coord_all.append(nuc_coord)
                try:
                    skel = pcg_skel.pcg_skeleton(root_id=latest_roots_PC[i], client=client, root_point=nuc_coord, collapse_soma=False, collapse_radius=50)
                    meshparty.skeleton_io.export_to_swc(skel, './skel/pyc_with_axon/pyc_' + str(i) + '.swc', node_labels=None, radius=None, 
                                                    header=None, xyz_scaling=1000, resample_spacing=500, interp_kind='linear', 
                                                    tip_length_ratio=0.25, avoid_root=True)
                    #meshparty.skeleton_io.export_to_swc(skel, './skel/pyc_with_axon/pyc_' + str(i) + '.swc', node_labels=None, radius=None, 
                    #                                header=None, xyz_scaling=1000, tip_length_ratio=0.25, avoid_root=True)
                    print(i)
                except Exception as e:
                    print('skipping iteration ' + str(i))
                    continue
            

In [ ]:
def split_skeleton(graph, split_node, nuclei_node):
    graph.remove_node(split_node)
    ccs = list(nx.connected_components(graph))
    subgraphs = [graph.subgraph(cc).copy() for cc in ccs]
    if len(subgraphs) >= 2:        
        if nuclei_node in subgraphs[0]:        
            return subgraphs[1], subgraphs[0]
        elif nuclei_node in subgraphs[1]:
            return subgraphs[0], subgraphs[1]
        else:
            print('cannot select the axon graph')            
    else:
        print('skeleton split failed.... i= ' + str(i))
        return subgraphs[0], None

def get_nearest_skel_node(skel, p):
    skel['distance'] = np.sqrt(
        (skel['x'] - p[0]) ** 2 +
        (skel['y'] - p[1]) ** 2 +
        (skel['z'] - p[2]) ** 2
    )
    nearest_node = skel.loc[skel['distance'].idxmin()]
    return nearest_node

def euc_distance(n1,n2):
    return np.sqrt((n1['x'] - n2['x'])**2 + (n1['y']-n2['y'])**2 + (n1['z'] - n2['z'])**2)


columns = ['id','type','x','y','z','radius','parent']
#axon_lengths = []
#axon_lengths.append(-1)
for i in range(846, len(pyc_i_with_ais)):
    if pyc_i_with_ais[i] in [1019,1060,1374,1403,1641,1619,1635,1718,1806,1820]:   # wrong AIS and minimal axon
        axon_lengths.append(-1)
    else:
        j = pyc_i_with_ais[i]
        sk = pd.read_csv('./skel/pyc_with_axon/pyc_' + str(j) + '.swc', delim_whitespace=True, names=columns)
        G = nx.Graph()
        for index, row in sk.iterrows():
            G.add_node(row['id'], pos=(row['x'], row['y'], row['z']))
        for index, row in sk.iterrows():
            if row['parent'] != -1:
                node1 = sk.loc[sk['id'] == row['id']].iloc[0]
                node2 = sk.loc[sk['id'] == row['parent']].iloc[0]
                dist = euc_distance(node1, node2)            
                G.add_edge(row['id'], row['parent'], weight=dist)
        ais_node_id = get_nearest_skel_node(sk, ais_all[i])
        nuc_node_id = get_nearest_skel_node(sk, nuc_coord_all[i])
        subgraph_axon, subgraph_dend = split_skeleton(G, ais_node_id['id'], nuc_node_id['id'])
        axon_nodes = subgraph_axon.nodes
        axon_df = sk[sk['id'].isin(axon_nodes)]
        axon_len = subgraph_axon.size(weight='weight')
        if subgraph_dend == None:
            axon_lengths.append(-1)
        else:
            axon_lengths.append(axon_len)
        scatter1=go.Scatter3d(x=sk['x'],y=sk['y'],z=sk['z'],mode='markers', marker=dict(size=2))
        scatter2=go.Scatter3d(x=axon_df['x'],y=axon_df['y'],z=axon_df['z'],mode='markers', marker=dict(size=2))
        scatter_ais = go.Scatter3d(x=[ais_all[i][0]], y=[ais_all[i][1]], z=[ais_all[i][2]], mode='markers', marker=dict(size=5, color='tomato'))
        layout = go.Layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z',aspectmode="data"))
        fig = go.Figure(data=[scatter1,scatter2, scatter_ais], layout=layout)
        pio.write_image(fig, './skel/pyc_with_axon/fig/pyc_'+str(pyc_i_with_ais[i])+'_'+str(int(axon_len))+'.jpeg', format='jpeg', scale=2)
        print(i)



In [ ]:
# Get AIS coordinates and write to spreadsheet 


import time
from nglui import parser

pyc_i_with_ais = []
ais_all = []
ais_error= []

#for i in range(0,1):
for i in range(0, len(latest_roots_PC)):
    if i % 100 == 0:
        print(i)
    #if i not in [190,284,883,1438]:
    if i not in [793]:
        link = pc_df['Final Link'][i]
        time.sleep(0.1)
        if link != '':
            this_state_id = link.split('/')[-1]
            json_info = client.state.get_state_json(state_id =this_state_id)
            layerNames = parser.layer_names(json_info)
            ais_tab = [i for i, s in enumerate(layerNames) if "AIS" in s]
            try:
                pts, tags = parser.point_annotations(json_info, layerNames[ais_tab[0]], tags=True)
            except:
                ais_error.append(i+2)
                ais_all.append(['NA','NA','NA'])
                continue        
                
            if pts != []:
                pyc_i_with_ais.append(i)
                ais = [round(pts[0][0] /18 * 4), round(pts[0][1] /18 * 4), round(pts[0][2] /45 * 40)]
                ais_all.append(ais)
            else:
                ais_all.append(['NA','NA','NA'])
        else:
            ais_all.append(['NA','NA','NA'])
    else:
        ais_all.append(['NA','NA','NA'])
            

#worksheet.update_values('AI2', ais_all)


In [ ]:
worksheet.update_values('AI2', ais_all)


In [ ]:
# axon length failures

pyc_i_failed = [29,219,245,370,557,561,603,646,747,792,796,823,830,910,958,962,1374,1404,1408,
                1410,1418,1420,1434,1479,1487,1505,1523,1526,1542,1554,1566,1572,1575,1595,1617,1638,
                1641,1647,1651,1661,1663,1673,1684,1718,1728,1753,1763,1802,1807,1811,1837]

pyc_i_manual = [29,561,747,792,823,830,
                910,958,1374,1404,1408,1415,1418,1420,1434,1479,1487,1505,
                1554,1566,1572,1575,1595,1617,1638,1641,1661,1673,
                1684,1728,1753,1763,1765,1794,1807,1811,1837]


ais_manual = [[44879,52318,832],[42444,46893,1307],[54664,52458,1541],[57534,50874,893],[46066,48031,1331],[61540,58088,939],
             [60316,65507,1386],[43150,45624,851],[28464,71694,675],[41238, 44676, 1065],[31790, 63987, 1060],[62720, 64129, 1360],[48812,47481,1166],[39731, 43786, 831],[53616, 49740, 1083],[34234,51794,599],[53567, 47324, 924],[61290,71906,846],
            [54480,45982,743],[43197, 45603, 916],[31765, 60128, 831],[56072,52033,760],[53682, 48715, 1192],[30323, 64754, 821],[61157, 62726, 814],[50536, 41175, 627],[39767, 44799, 1022],[40726, 47235, 773],
                [30285, 72805, 1090],[57200, 52433, 578],[53401, 48812, 901],[33538, 51888, 474],[33619, 51964,423],[52453, 46064, 550],[42942, 45535, 904],[56855, 52544,511],[49155, 38795, 670]
             ]

pyc_i_axon_self_touch = [245,370,557,796,962,1410,1523,1526,1651,1663,1753,1794]

In [ ]:
print(set(pyc_i_manual) - set(pyc_i_with_ais))

In [ ]:
import copy

axon_lengths_copy = copy.deepcopy(axon_lengths)
axon_lengths_additional = []

for i in range(0, len(pyc_i_manual)):
    j = pyc_i_manual[i]
    idxx = pyc_i_with_ais.index(j)
    sk = pd.read_csv('./skel/pyc_with_axon/pyc_' + str(j) + '.swc', delim_whitespace=True, names=columns)
    G = nx.Graph()
    for index, row in sk.iterrows():
        G.add_node(row['id'], pos=(row['x'], row['y'], row['z']))

    for index, row in sk.iterrows():
        if row['parent'] != -1:
            node1 = sk.loc[sk['id'] == row['id']].iloc[0]
            node2 = sk.loc[sk['id'] == row['parent']].iloc[0]
            dist = euc_distance(node1, node2)            
            G.add_edge(row['id'], row['parent'], weight=dist)

    ais_coord = np.array(ais_manual[i]) * np.array([0.018,0.018,0.045])
    nuc_coord = np.array(nuclei_coord_in_vol[j,:])* np.array([0.018,0.018,0.045])
    ais_node_id = get_nearest_skel_node(sk, np.array(ais_coord))
    nuc_node_id = get_nearest_skel_node(sk, nuc_coord)
    subgraph_axon, subgraph_dend = split_skeleton(G, ais_node_id['id'], nuc_node_id['id'])
    axon_nodes = subgraph_axon.nodes
    axon_df = sk[sk['id'].isin(axon_nodes)]
    axon_len = subgraph_axon.size(weight='weight')
    if subgraph_dend == None:
        axon_lengths_additional.append(-1)
        axon_lengths[idxx] = -1
    else:
        axon_lengths_additional.append(axon_len)
        axon_lengths[idxx] = axon_len
    scatter1=go.Scatter3d(x=sk['x'],y=sk['y'],z=sk['z'],mode='markers', marker=dict(size=2))
    scatter2=go.Scatter3d(x=axon_df['x'],y=axon_df['y'],z=axon_df['z'],mode='markers', marker=dict(size=2))
    scatter_ais = go.Scatter3d(x=[ais_coord[0]], y=[ais_coord[1]], z=[ais_coord[2]], mode='markers', marker=dict(size=5, color='tomato'))
    layout = go.Layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z',aspectmode="data"))
    fig = go.Figure(data=[scatter1,scatter2, scatter_ais], layout=layout)
    pio.write_image(fig, './skel/pyc_with_axon/fig/pyc_'+str(j)+'_'+str(int(axon_len))+'.jpeg', format='jpeg', scale=2)


In [ ]:
unable_to_rescue = [910,1374,1415,1728,1765,1794,1802]
for i in range(0, len(unable_to_rescue)):
    j = unable_to_rescue[i]
    idxx = pyc_i_with_ais.index(j)
    axon_lengths[idxx] = -1


In [ ]:
# edit pandas dataframe

print(len(pyc_i_with_ais))
print(len(axon_lengths))
pc_df['axon_length'] = None
pc_df.loc[pyc_i_with_ais, 'axon_length'] = axon_lengths


In [ ]:
# additional manual annotation AIS (old version)

index_added = [510,518,552,564,792,1641,1661,1663,1763,860,883,933,941,955,968,976,982,990]
ais_added = [[31808,68874,1474],[55284,55093,1362],[60460,61665,1321],[31088,62532,1424],[57534,50862,923],[50993,41160,917],[39669,44951,1064],[56296,51276,1363],[33606,51896,492],[30601,62568,1161],[59570,54184,1032],[47299,37349,894],[35243,46947,783],[53415,39951,818],[43186,35865,981],[39125,42407,730],[47628,37848,660],[47628,37848,660]]

pyc_i_with_ais2 = []
ais_all2 = []
nuc_coord_all2 = []
ais_error2= []

for i in range(0, len(index_added)):
    pyc_i_with_ais2.append(index_added[i])
    pts = ais_added[i]
    ais = np.array([pts[0], pts[1], pts[2]]) * np.array([0.018,0.018,0.045])
    ais_all2.append(ais)
    nuc_coord = np.array(nuclei_coord_in_vol[index_added[i],:])* np.array([0.018,0.018,0.045])
    nuc_coord_all2.append(nuc_coord)
    try:
        skel = pcg_skel.pcg_skeleton(root_id=latest_roots_PC[index_added[i]], client=client, root_point=nuc_coord, collapse_soma=False, collapse_radius=50)
        meshparty.skeleton_io.export_to_swc(skel, './skel/pyc_with_axon/pyc_' + str(index_added[i]) + '.swc', node_labels=None, radius=None, 
                                        header=None, xyz_scaling=1000, resample_spacing=500, interp_kind='linear', 
                                        tip_length_ratio=0.25, avoid_root=True)
        print(i)
    except Exception as e:
        print('skipping iteration ' + str(i))
        continue

columns = ['id','type','x','y','z','radius','parent']
axon_lengths_additional = []
#for i in range(0, 1):
for i in range(0, len(index_added)):
    j = index_added[i]
    sk = pd.read_csv('./skel/pyc_with_axon/pyc_' + str(j) + '.swc', delim_whitespace=True, names=columns)
    G = nx.Graph()
    for index, row in sk.iterrows():
        G.add_node(row['id'], pos=(row['x'], row['y'], row['z']))

    for index, row in sk.iterrows():
        if row['parent'] != -1:
            node1 = sk.loc[sk['id'] == row['id']].iloc[0]
            node2 = sk.loc[sk['id'] == row['parent']].iloc[0]
            dist = euc_distance(node1, node2)            
            G.add_edge(row['id'], row['parent'], weight=dist)
    ais_node_id = get_nearest_skel_node(sk, np.array(ais_added[i]) * np.array([0.018,0.018,0.045]))
    nuc_node_id = get_nearest_skel_node(sk, nuc_coord_all2[i])
    subgraph_axon, subgraph_dend = split_skeleton(G, ais_node_id['id'], nuc_node_id['id'])
    axon_nodes = subgraph_axon.nodes
    axon_df = sk[sk['id'].isin(axon_nodes)]
    axon_len = subgraph_axon.size(weight='weight')
    if subgraph_dend == None:
        axon_lengths_additional.append(-1)
    else:
        axon_lengths_additional.append(axon_len)
    scatter1=go.Scatter3d(x=sk['x'],y=sk['y'],z=sk['z'],mode='markers', marker=dict(size=2))
    scatter2=go.Scatter3d(x=axon_df['x'],y=axon_df['y'],z=axon_df['z'],mode='markers', marker=dict(size=2))
    scatter_ais = go.Scatter3d(x=[ais_all2[i][0]], y=[ais_all2[i][1]], z=[ais_all2[i][2]], mode='markers', marker=dict(size=5, color='tomato'))
    layout = go.Layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z',aspectmode="data"))
    fig = go.Figure(data=[scatter1,scatter2, scatter_ais], layout=layout)
    print(i)
    pio.write_image(fig, './skel/pyc_with_axon/fig/pyc_'+str(index_added[i])+'_'+str(int(axon_len))+'.jpeg', format='jpeg', scale=2)


In [ ]:
index_added = [510,518,552,564,792,1641,1661,1663,1763,860,883,933,941,955,968,976,982,990]

print(len(index_added))
print(axon_lengths_additional)

In [ ]:
# Find cells with incorrect AIS annotation (i.e. multiply annotated)

gc2 = pygsheets.authorize(service_file = "./hippca3-8126bea0d603.json")
sheet2 = gc2.open('CA3 proofreading')
worksheet2 = sheet2.worksheet('title','1.pyramidal_cell_list')
pc_df2 = worksheet2.get_as_df(start='A9')
pc_df2 = pc_df2.iloc[:-2]

nuclei_coord_in_vol = pc_df2[['x','y','z']].to_numpy()
latest_roots_PC2 = []
print('Getting latest roots for PC...')  
cell_ids_scatter = vol.scattered_points(nuclei_coord_in_vol)
for i in range(0, len(nuclei_coord_in_vol)):
    if i % 300 == 0:
        print(i)
    cell_ID = [cell_ids_scatter.get(tuple(nuclei_coord_in_vol[i,:]))]
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0]-150, nuclei_coord_in_vol[i,1], nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0], nuclei_coord_in_vol[i,1]+150, nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0], nuclei_coord_in_vol[i,1]-150, nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        print("Failed to find neuron segment ID using nuclei center")
    else:
        if client.chunkedgraph.is_latest_roots(cell_ID[0]):
            latest_roots_PC2.append(cell_ID[0])
        else:
            latest_roots_PC2.append(client.chunkedgraph.suggest_latest_roots(cell_ID[0]))
latest_roots_PC_str2 = [str(num) for num in latest_roots_PC2]

worksheet3 = sheet2.worksheet('title','6.Additional_pyramidal')
pc_df3 = worksheet3.get_as_df(start='A9')
pc_df3 = pc_df3.iloc[:-2]

nuclei_coord_in_vol = pc_df3[['x','y','z']].to_numpy()
latest_roots_PC3 = []
print('Getting latest roots for PC...')  
cell_ids_scatter = vol.scattered_points(nuclei_coord_in_vol)
for i in range(0, len(nuclei_coord_in_vol)):
    if i % 300 == 0:
        print(i)
    cell_ID = [cell_ids_scatter.get(tuple(nuclei_coord_in_vol[i,:]))]
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0]-150, nuclei_coord_in_vol[i,1], nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0], nuclei_coord_in_vol[i,1]+150, nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        cell_ID = np.array([vol[nuclei_coord_in_vol[i,0], nuclei_coord_in_vol[i,1]-150, nuclei_coord_in_vol[i,2]].squeeze()])
    if cell_ID[0] == 0:
        print("Failed to find neuron segment ID using nuclei center")
    else:
        if client.chunkedgraph.is_latest_roots(cell_ID[0]):
            latest_roots_PC3.append(cell_ID[0])
        else:
            latest_roots_PC3.append(client.chunkedgraph.suggest_latest_roots(cell_ID[0]))
latest_roots_PC_str3 = [str(num) for num in latest_roots_PC3]
print('---------------')

for i in range(0, len(latest_roots_PC2)):
    if i not in [293,1269]:
        link = pc_df2['Final Link'][i]
        time.sleep(0.2)
        if link != '':
            this_state_id = link.split('/')[-1]
            json_info = client.state.get_state_json(state_id =this_state_id)
            layerNames = parser.layer_names(json_info)
            ais_tab = [i for i, s in enumerate(layerNames) if "AIS" in s]
            try:
                pts, tags = parser.point_annotations(json_info, layerNames[ais_tab[0]], tags=True)
            except:
                ais_error.append(i+2)
                continue
            if len(pts) > 1:
                print(pc_df2['segid (Nuclei table segID)'].values[i])

print('---------------')            

for i in range(0, len(latest_roots_PC3)):
    link = pc_df3['Final Link'][i]
    time.sleep(0.2)
    if link != '':
        this_state_id = link.split('/')[-1]
        json_info = client.state.get_state_json(state_id =this_state_id)
        layerNames = parser.layer_names(json_info)
        ais_tab = [i for i, s in enumerate(layerNames) if "AIS" in s]
        try:
            pts, tags = parser.point_annotations(json_info, layerNames[ais_tab[0]], tags=True)
        except:
            ais_error.append(i+2)
            continue
        if len(pts) > 1:
            print(pc_df3['segid (Nuclei table segID)'].values[i])
                


In [ ]:
sk3 = pd.read_csv('./test_skeletor.swc', delim_whitespace=True, names=columns, skiprows=6)
sk3[['x','y','z']] = sk3[['x','y','z']] / 1000
print(sk3)
#axon_nodes = subgraph_axon.nodes
#axon_df = sk[sk['id'].isin(axon_nodes)]
scatter11=go.Scatter3d(x=sk3['x'],y=sk3['y'],z=sk3['z'],mode='markers', marker=dict(size=2))
#scatter12=go.Scatter3d(x=axon_df['x'],y=axon_df['y'],z=axon_df['z'],mode='markers', marker=dict(size=2))
scatter_ais = go.Scatter3d(x=[ais_all[0][0]], y=[ais_all[0][1]], z=[ais_all[0][2]], mode='markers', marker=dict(size=10, color='tomato'))
scatter_nuc = go.Scatter3d(x=[nuc_coord_all[0][0]], y=[nuc_coord_all[0][1]], z=[nuc_coord_all[0][2]], mode='markers', marker=dict(size=20, color='violet'))
layout = go.Layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z',aspectmode="data"))
fig = go.Figure(data=[scatter11, scatter_ais, scatter_nuc], layout=layout)
fig.show()